This notebook is used to construct matrix form game from network and apply best response algorithm. 

### Case 1； 4x4

In [1]:
import numpy as np
import random

In [2]:
graph_cost = {
    "A": {"B":lambda x : x, "C": lambda x : 2*x},
    "B": {"D":lambda x: 99*x -99, "E": lambda x : x},
    "C": {"D":lambda x: 2*x + 5, "E": lambda x: 2*x},
    "D": {"F": lambda x: 3},
    "E": {"F": lambda x: 5},
    "F": []
}

Graph = {'A':  ['B', 'C'], # 构建树
        'B':  ['D', 'E'],
        'C':  ['D', 'E'],
        'D':  ['F'],
        'E':  ['F'],
        'F':  []
         }

In [3]:
def generatePath(graph, path, end, results): # 生成路径
    state = path[-1]
    if state == end:
        results.append(path)
    else:
        for arc in graph[state]:
            if arc not in path: 
                generatePath(graph, path + [arc], end, results)

In [4]:
def searchGraph(graph, start, end):					# 搜索树
	results = []                
	generatePath(graph, [start], end, results)			# 生成路径
	results.sort(key=lambda x:len(x))					# 按路径长短排序
	return results
def generatePath(graph, path, end, results):			# 生成路径
    state = path[-1]
    if state == end:
        results.append(path)
    else:
        for arc in graph[state]:
            if arc not in path: 
                generatePath(graph, path + [arc], end, results)

In [5]:
def generate_link(r):
    link = []
    for i in range(len(r)):
        path = r[i]
        link_perpath = []
        for j in range(0,len(r[0])-1):
            link_perpath.append([path[j], path[j+1]])
        link.append(link_perpath)
    return link

def generate_payoff(r, link): 
    num_strategy = 2**(len(r[0])-2)
    num_agent = 2
    cost_p1 = np.zeros([num_strategy, num_strategy])
    cost_p2 = np.zeros([num_strategy, num_strategy])
    cost1 = 0
    cost2 = 0
    for p1 in range(len(r)):
        path_p1 = link[p1]
        for p2 in range(len(r)):
            path_p2 = link[p2]
#             print(path_p2)
            for j in range(0, len(r[0])-1):
                if path_p1[j] == path_p2[j]:
                    x = 2
                else:
                    x = 1 
#                 print(p1, p2, x, graph_cost[path_p1[j][0]][path_p1[j][1]](x))
                cost1 += graph_cost[path_p1[j][0]][path_p1[j][1]](x)
                cost2 += graph_cost[path_p2[j][0]][path_p2[j][1]](x)
            cost_p1[p1, p2] = cost1
            cost_p2[p1, p2] = cost2
            cost1 = 0
            cost2 = 0
            
    Cost_p1 = (-cost_p1) #.tolist()
    Cost_p2 = (-cost_p2) #.tolist()
    
    # payoff matrix
    payoff_matrix = np.zeros([num_players, num_strategy, num_strategy])

    payoff_matrix[0] = Cost_p1
    payoff_matrix[1] = Cost_p2

    return Cost_p1, Cost_p2, payoff_matrix

In [6]:
def best_response(num_iter, r, num_players):
    '''
    this is best response dynamic demo
    return: NE
    '''
    # number of iterations
    
    num_strategy = 2**(len(r[0])-2)

    # 'strategy' stores each players' mixed strategy (probability of action 1 in each case)
    strategys = np.zeros([num_players, num_iter, num_strategy])

    payoffs = np.zeros([num_strategy, num_players]) 

    # random initialization p1 p2
    
    m = np.zeros([2])
    m[0] = random.randint(0,3)
    strategys[0][0, int(m[0])] = 1
    m[1] = random.randint(0,3)
    strategys[1][0, int(m[1])] = 1
    
    '''
    m = np.zeros([2])
    m[0] = 0
    strategys[0][0, int(m[0])] = 1
    m[1] = 0
    strategys[1][0, int(m[1])] = 1
    '''

    payoff_temp = np.zeros([num_strategy,num_strategy])

    for i in range(1,num_iter):

        if i % 2: # 1,3,5,7 ...
    #         print(strategys[1][i-1, :].reshape(1,num_strategy).shape)
    #         print(payoff_matrix[0].shape)
            payoff_temp = strategys[1][i-1, :].reshape(1,num_strategy) * payoff_matrix[0]
            payoffs[:,0] = payoff_temp[:, int(m[1])]

            if max(payoffs[:, 0]) > np.sum(payoffs[:,0] * strategys[0][i-1,:]):
                strategys[0][i, np.argmax(payoffs[:,0])] = 1
            else:
                strategys[0][i, :] = strategys[0][i-1,:]
            strategys[1][i, :] = strategys[1][i-1,:]

        else: 
            payoff_temp = strategys[0][i-1, :].reshape(num_strategy,1) * payoff_matrix[1]
            payoffs[:,1] = payoff_temp[int(m[0]),:]

            if max(payoffs[:, 1]) > np.sum(payoffs[:,1] * strategys[1][i-1,:]):
                strategys[1][i, np.argmax(payoffs[:,1])] = 1
            else:
                strategys[1][i, :] = strategys[1][i-1,:]
            strategys[0][i, :] = strategys[0][i-1,:]

        # update m
        m[0] = np.where(strategys[0][i, :] == 1)[0]
        m[1] = np.where(strategys[1][i, :] == 1)[0]
    return strategys

In [11]:
r = searchGraph(Graph, 'A','F') # 搜索A到D的所有路径
print('************************')
print('     path A to F')
print('************************')
for i in r:
    print(i)

************************
     path A to F
************************
['A', 'B', 'D', 'F']
['A', 'B', 'E', 'F']
['A', 'C', 'D', 'F']
['A', 'C', 'E', 'F']


In [12]:
r = searchGraph(Graph, 'A','F')
link = generate_link(r)
link

[[['A', 'B'], ['B', 'D'], ['D', 'F']],
 [['A', 'B'], ['B', 'E'], ['E', 'F']],
 [['A', 'C'], ['C', 'D'], ['D', 'F']],
 [['A', 'C'], ['C', 'E'], ['E', 'F']]]

In [13]:
_,_, payoff_matrix = generate_payoff(r, link)
payoff_matrix

array([[[-104.,   -5.,   -4.,   -4.],
        [  -8.,   -9.,   -7.,   -7.],
        [ -12.,  -12.,  -16.,  -14.],
        [  -9.,   -9.,  -11.,  -13.]],

       [[-104.,   -8.,  -12.,   -9.],
        [  -5.,   -9.,  -12.,   -9.],
        [  -4.,   -7.,  -16.,  -11.],
        [  -4.,   -7.,  -14.,  -13.]]])

In [14]:
num_iter = 5
num_players = 2
strategys = best_response(num_iter, r, num_players)
strategys

array([[[0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]]])

best response: demo

### Case 2: 8x8

In [15]:
graph_cost = {
    "A": {"B":lambda x : x, "C": lambda x : 2*x},
    "B": {"D":lambda x: 99*x -99, "E": lambda x : x},
    "C": {"D":lambda x: 2*x + 5, "E": lambda x: 2*x},
    "D": {"G": lambda x: 2 + x, 'H': lambda x: 3*x},
    "E": {"G": lambda x: x, 'H': lambda x: 5*x},
    "G": {"F": lambda x: 2*x},
    "H": {"F": lambda x: 5},
    "F": []
}

Graph = {'A':  ['B', 'C'], # 构建树
        'B':  ['D', 'E'],
        'C':  ['D', 'E'],
        'D':  ['G', 'H'],
        'E':  ['G', 'H'],
        'G': ['F'],
        'H': ['F'],
        'F':  []
         }

In [16]:
r = searchGraph(Graph, 'A','F')
link = generate_link(r)
# print(link)
_,_, payoff_matrix = generate_payoff(r, link)
payoff_matrix

array([[[-109., -106.,   -9.,   -7.,   -9.,   -6.,   -8.,   -6.],
        [-109., -112.,  -10.,  -10.,   -9.,  -12.,   -9.,   -9.],
        [  -8.,   -6.,  -10.,   -7.,   -7.,   -5.,   -8.,   -5.],
        [ -13.,  -13.,  -14.,  -19.,  -12.,  -12.,  -12.,  -17.],
        [ -17.,  -14.,  -16.,  -14.,  -21.,  -18.,  -18.,  -16.],
        [ -17.,  -20.,  -17.,  -17.,  -21.,  -24.,  -19.,  -19.],
        [  -9.,   -7.,  -10.,   -7.,  -11.,   -9.,  -14.,  -11.],
        [ -14.,  -14.,  -14.,  -19.,  -16.,  -16.,  -18.,  -23.]],

       [[-109., -109.,   -8.,  -13.,  -17.,  -17.,   -9.,  -14.],
        [-106., -112.,   -6.,  -13.,  -14.,  -20.,   -7.,  -14.],
        [  -9.,  -10.,  -10.,  -14.,  -16.,  -17.,  -10.,  -14.],
        [  -7.,  -10.,   -7.,  -19.,  -14.,  -17.,   -7.,  -19.],
        [  -9.,   -9.,   -7.,  -12.,  -21.,  -21.,  -11.,  -16.],
        [  -6.,  -12.,   -5.,  -12.,  -18.,  -24.,   -9.,  -16.],
        [  -8.,   -9.,   -8.,  -12.,  -18.,  -19.,  -14.,  -18.],
        

In [17]:
num_iter = 5
num_players = 2
strategys = best_response(num_iter, r, num_players)
strategys

array([[[0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.]]])

Case 3:4x4; 2 NE

In [18]:
graph_cost = {
    "A": {"B":lambda x : x, "C": lambda x : 3},
    "B": {"D":lambda x: 1, "E": lambda x : x},
    "C": {"D":lambda x: 1, "E": lambda x: x },
    "D": {"F": lambda x: x},
    "E": {"F": lambda x: 2},
    "F": []
}

Graph = {'A':  ['B', 'C'], # 构建树
        'B':  ['D', 'E'],
        'C':  ['D', 'E'],
        'D':  ['F'],
        'E':  ['F'],
        'F':  []
         }

In [19]:
r = searchGraph(Graph, 'A','F')
link = generate_link(r)
_,_, payoff_matrix = generate_payoff(r, link)
num_iter = 5
num_players = 2
strategys = best_response(num_iter, r, num_players)
strategys # (1,2) (1,3) (4,1)

array([[[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]],

       [[0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]]])

In [20]:
payoff_matrix

array([[[-5., -4., -4., -3.],
        [-5., -6., -4., -4.],
        [-6., -5., -6., -5.],
        [-6., -6., -6., -7.]],

       [[-5., -5., -6., -6.],
        [-4., -6., -5., -6.],
        [-4., -4., -6., -6.],
        [-3., -4., -5., -7.]]])

In [21]:
graph_cost = {
    "A": {"B":lambda x : x, "C": lambda x : 3},
    "B": {"D":lambda x: 10, "E": lambda x : 5*x},
    "C": {"D":lambda x: x, "E": lambda x: x},
    "D": {"G": lambda x: 1, 'H': lambda x: x},
    "E": {"G": lambda x: 1, 'H': lambda x: x},
    "G": {"F": lambda x: 5},
    "H": {"F": lambda x: x},
    "F": []
}

Graph = {'A':  ['B', 'C'], # 构建树
        'B':  ['D', 'E'],
        'C':  ['D', 'E'],
        'D':  ['G', 'H'],
        'E':  ['G', 'H'],
        'G': ['F'],
        'H': ['F'],
        'F':  []
         }

In [22]:
r = searchGraph(Graph, 'A','F')
link = generate_link(r)
_,_, payoff_matrix = generate_payoff(r, link)
num_iter = 5
num_players = 2
strategys = best_response(num_iter, r, num_players)
strategys

array([[[0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1.]]])

In [23]:
payoff_matrix

array([[[-18., -18., -18., -18., -17., -17., -17., -17.],
        [-14., -16., -14., -15., -13., -15., -13., -14.],
        [-13., -13., -18., -18., -12., -12., -12., -12.],
        [ -9., -10., -14., -16.,  -8.,  -9.,  -8., -10.],
        [-10., -10., -10., -10., -11., -11., -10., -10.],
        [ -6.,  -8.,  -6.,  -7.,  -7.,  -9.,  -6.,  -7.],
        [-10., -10., -10., -10., -10., -10., -11., -11.],
        [ -6.,  -7.,  -6.,  -8.,  -6.,  -7.,  -7.,  -9.]],

       [[-18., -14., -13.,  -9., -10.,  -6., -10.,  -6.],
        [-18., -16., -13., -10., -10.,  -8., -10.,  -7.],
        [-18., -14., -18., -14., -10.,  -6., -10.,  -6.],
        [-18., -15., -18., -16., -10.,  -7., -10.,  -8.],
        [-17., -13., -12.,  -8., -11.,  -7., -10.,  -6.],
        [-17., -15., -12.,  -9., -11.,  -9., -10.,  -7.],
        [-17., -13., -12.,  -8., -10.,  -6., -11.,  -7.],
        [-17., -14., -12., -10., -10.,  -7., -11.,  -9.]]])

In [24]:
graph_cost = {
    "A": {"B":lambda x : x, "C": lambda x : 3},
    "B": {"D":lambda x: 1, "E": lambda x : x},
    "C": {"D":lambda x: 1, "E": lambda x: x },
    "D": {"F": lambda x: x},
    "E": {"F": lambda x: 2},
    "F": []
}

Graph = {'A':  ['B', 'C'], # 构建树
        'B':  ['D', 'E'],
        'C':  ['D', 'E'],
        'D':  ['F'],
        'E':  ['F'],
        'F':  []
         }